## PDF bi-variate normal plot

In [ ]:
import logging

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

import scipy.stats as sts
from scipy.stats import norm
from scipy.stats import multivariate_normal

from synthsonic.models.kde_copula_nn_pdf import KDECopulaNNPdf

In [ ]:
import ppu
from ppu.viz import plot_dense_scatter, plot_ellipse_from_cov, plot_pdf_contours

## Config

In [ ]:
colors = ppu.viz.set_plot_style(True)

In [ ]:
def _get_32bit_seed(rng):
    return rng.integers(low=0, high=np.iinfo(np.uint32).max, size=1, dtype=np.uint32)[0]

In [ ]:
rng = np.random.Generator(np.random.PCG64DXSM(42))

## Data

In [ ]:
n_samples = 100_000

Generate bivariate gaussian with correlation

In [ ]:
bg_gen = ppu.generator.BivariateGaussian(cov=0.7, rng=rng)

In [ ]:
X = bg_gen.rvs(n_samples)

Plot with the samples and the theoretical distribution

In [ ]:
ax = bg_gen.plot(filled=False)
ax = plot_dense_scatter(X, ax, color=colors[2])
ax.set_title("samples Bi-variate standard normal")

## Fit

Fit the DGP

In [ ]:
pdf = KDECopulaNNPdf(random_state=_get_32bit_seed(rng))
pdf = pdf.fit(X)

From the fitted DGP sample points

In [ ]:
X_gen = pdf.sample_no_weights(n_samples, random_state=_get_32bit_seed(rng))

Plot generated points over expected joint distribution

In [ ]:
ax = ax = bg_gen.plot(filled=False)
ax = plot_dense_scatter(X_gen, ax)
ax.set_title("$X_{gen}$ -- $X \sim $Bivariate")

Fitted vs DGP

In [ ]:
ax = ax = bg_gen.plot(filled=False)
ax = plot_dense_scatter(X_gen, ax)
ax = plot_dense_scatter(X, ax, color=colors[2])
ax.set_title("$X_{gen}$ (red) -- $X$ (Blue) -- $X \sim $Bivariate")

In [ ]:
def get_scaled_linspace(x, factor: float = 0.1):
    return np.sort(X[:, 0])[[0, -1]] * np.array((1.0 - factor, 1.0 + factor))

In [ ]:
n_bins = 500

In [ ]:
x = np.linspace(*get_scaled_linspace(X[:, 0]), n_bins)
y = np.linspace(*get_scaled_linspace(X[:, 1]), n_bins)

xs, ys = np.meshgrid(x, y)
X_grid = np.c_[xs.ravel(), ys.ravel()]

In [ ]:
mean_xy = X.mean(0)
sample_cov = np.cov(X, rowvar=False, ddof=1)

In [ ]:
dgp_x_grid = bg_gen.dist.pdf(X_grid).reshape((n_bins, n_bins))

In [ ]:
pdf_x_grid = pdf.pdf(X_grid).reshape((n_bins, n_bins))

In [ ]:
ref_pdf = pdf.pdf(X)

Compare estimated PDF vs DGP

In [ ]:
levels = (0.90, 0.95, 0.975)
fig, ax = plt.subplots(figsize=(12, 8))
ax = bg_gen.plot(levels=levels, ax=ax)
ax = plot_pdf_contours(xs, ys, pdf_x_grid, ref_pdf, ax=ax, levels=levels)
_ = ax.set_title("$\widehat{PDF}_{X}$ -- $X \sim $Bivariate")

In [ ]:
levels = (0.9, 0.95, 0.99, 0.999)
fig, ax = plt.subplots(figsize=(12, 8))
ax = bg_gen.plot(levels=levels, ax=ax)
ax = plot_pdf_contours(xs, ys, pdf_x_grid, ref_pdf, ax=ax, levels=levels)
_ = ax.set_title("$\widehat{PDF}_{X}$ -- $X \sim $Bivariate")

In [ ]:
levels = (0.90, 0.95, 0.99, 0.99, 0.999)
fig, ax = plt.subplots(figsize=(12, 8))
ax = bg_gen.plot(ax=ax, levels=levels)
ax = plot_pdf_contours(xs, ys, dgp_x_grid, bg_gen.dist.pdf(X), ax=ax, levels=levels)
_ = ax.set_title("$PDF_{X}$ -- $X \sim $Bivariate")

## Validate fit

In [ ]:
assert False

In [ ]:
pdf._calibrate_classifier(pdf.hist_p0_, pdf.hist_p1_, pdf.bin_edges_, validation_plots=True)

In [ ]:
pdf.score(X)

In [ ]:
p = pdf.pdf(X)
logp = pdf.logpdf(X)

In [ ]:
# theoretical pdf values
p2 = bg_gen.dist.pdf(X)
logp2 = np.log(p2)

In [ ]:
X_gen = pdf.sample_no_weights(n_samples=X.shape[0])

In [ ]:
s_cov = np.round(np.cov(X_gen.T), 3)[0, 1]
s_mu = np.round(X_gen.mean(), 3)


print('mu_hat: ', s_mu)
print('cov_hat: ', s_cov)

In [ ]:
# compare the two
x = np.linspace(0, 0.223, 100)
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(x, x, lw=2, ls='--', zorder=10, color='black')
ax.scatter(p2, p, s=0.005, color=colors[0], marker='x')
ax.set_xlabel(r'$X$', fontsize=18)
ax.set_ylabel(r'$X_{\rm syn}$', fontsize=18)
ax.set_ylim(-0.03, 0.35)
ax.tick_params(labelsize=16)

In [ ]:
# compare the two
x = np.linspace(-12.5, -1.47, 100)
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(x, x, lw=3, ls='--', zorder=10, color='black')
ax.scatter(logp2, logp, s=2, color=colors[0])
ax.set_xlabel(r'$X$', fontsize=18)
ax.set_ylabel(r'$X_{\rm syn}$', fontsize=18)
ax.tick_params(labelsize=16)